<a href="https://colab.research.google.com/github/manegene/fcc_book_recommendation_knn/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [84]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-18 08:14:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M   151MB/s    in 0.2s    

2025-03-18 08:14:10 (151 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [169]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [170]:
# add your code here - consider creating a new cell for each section of code
# Filter users with at least 200 ratings and books with at least 100 ratings
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

In [171]:
# Merge books and ratings data
df = pd.merge(df_ratings, df_books, on='isbn')

In [213]:
print(df.head())
print(df.map(lambda x: 'The Secret Life of Bees' in str(x)).any().any())

     user        isbn  rating  \
0  277427  0060930535     0.0   
1  277427  0060934417     0.0   
2  277427  0061009059     9.0   
3  277427  0142001740     0.0   
4  277427  0312966091     0.0   

                                               title              author  
0                      The Poisonwood Bible: A Novel  Barbara Kingsolver  
1                                 Bel Canto: A Novel        Ann Patchett  
2  One for the Money (Stephanie Plum Novels (Pape...     Janet Evanovich  
3                            The Secret Life of Bees       Sue Monk Kidd  
4  Three To Get Deadly : A Stephanie Plum Novel (...     Janet Evanovich  
True


In [215]:
# Create a pivot table
book_pivot = df.pivot_table(index='title', columns='user', values='rating').fillna(0)
book_sparse = csr_matrix(book_pivot)


In [216]:
# Fit the KNN model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [217]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    if book not in book_pivot.index:
        return [book, []]

    book_index = book_pivot.index.get_loc(book)
    distances, indices = model.kneighbors(book_pivot.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = [[book_pivot.index[i], distances[0][j]] for j, i in enumerate(indices[0]) if i != book_index]


    return [book, recommended_books]

In [232]:
# Test the function
books = get_recommends("A Painted House")
print(books)

['A Painted House', [['The Brethren', np.float32(0.7510461)], ['The Firm', np.float32(0.7575308)], ['We Were the Mulvaneys', np.float32(0.77621484)], ['The Pelican Brief', np.float32(0.77797955)], ['The Chamber', np.float32(0.7891382)]]]


In [233]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("A Painted House")
  if recommends[0] != "A Painted House":
    test_pass = False
  recommended_books = ["The Firm", 'The Brethren', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      print(recommends[1][i][0])
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    #print(recommends)
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉
